Uso il file di train per l'addestramento, contiene 96 esempi etichettati, e dovrebbe essere bilanciato. alla label viene associata la vettorizzazione del rumor verificato.

Una volta fatto questo passo alla calssificazione dei 32 rumor del devSet.

Per la rumor retrieval potrei ordinare in base a quelli col contesto più forte e prendere i primi 5 che classifica come il rumor

Come classificatore è stato scelto NaiveBayes per la scarsità dei dati.

Apertura del file per il train

In [1]:
import json

with open('English_train_v3.json', 'r') as f:
    trainSet = json.load(f)

Apertura del file dev

In [2]:
with open('English_dev.json', 'r') as f:
    devSet = json.load(f)

VETTORIZZAZIONE

In [3]:
#Lista di tutti i rumor
listaTrain = []
for oggetto in trainSet:
    listaTrain.append(oggetto["rumor"])

In [4]:
dimTrain = len(listaTrain)

In [5]:
#Lista di tutti i rumor
listaTest = []
for oggetto in devSet:
    listaTest.append(oggetto["rumor"])

In [6]:
dimTest = len(listaTest)

Creazione di una lista unica da vettorizzare

In [7]:
listaDaVett = listaTrain + listaTest

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import heapq
import re

def vettorizzazione(rumors):
    
    vectorizer = TfidfVectorizer(use_idf=True,sublinear_tf=True,smooth_idf=True, stop_words=None)
    tfidf_matrix = vectorizer.fit_transform(rumors)

    return tfidf_matrix

In [9]:
listaVett = vettorizzazione(listaDaVett)

Separo le vettorizzazioni di training e di test

In [10]:
trainVett = listaVett[:-dimTest]
testVett = listaVett[-dimTest:]

In [11]:
X = trainVett

Y = []
for oggetto in trainSet:
    Y.append(oggetto["label"])

ADDESTRAMENTO E CLASSIFFICAZIONE

In [12]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()

model= NB.fit(X,Y)

In [13]:
listaPredictions = model.predict(testVett)

Estrazione delle label dal test

In [14]:
true_labels = []
for element in devSet:
    true_labels.append(element['label'])

Calcolo della Macro_F1 senza suare le loro storie

In [15]:
from sklearn.metrics import f1_score

f1_score(true_labels, listaPredictions, labels=["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"],average='macro')

0.2574074074074074

Classificazione non precisa, per evidence retrieval uso i cinque tweet col contesto più forte

In [16]:
def flat_list(lista):
    ris = []
    for elemento in lista.toarray():
        ris.extend(elemento)
    return ris

In [17]:
from scipy.spatial.distance import cosine
import numpy

listaDizionari = []
listaDaVett = []

for rumor,prediction in zip(devSet,listaPredictions):
    #Vado a fare la vettorizzazione del rumor e della timeline
    listaDaVett.append(rumor['rumor'])
    for evidence in rumor['timeline']:
        listaDaVett.append(evidence[2])
        
    #Vado a vettorizzare
    listaVett = vettorizzazione(listaDaVett)

    #Creo una coppia della timeline e ci aggiungo alla fine i relativi valori di coseno con il rumor
    vettRumor = listaVett[0]
    listaVettEvidence = listaVett[1:]
    listaEvidence = rumor['timeline']
    i = 0

    for evidence,vettEvidence in zip(listaEvidence,listaVettEvidence):
        evidence.append(cosine(numpy.transpose(flat_list(vettRumor)),numpy.transpose(flat_list(vettEvidence))))

    #Ordino dal più basso al più alto che più è basso l'angolo e meglio è
    listaEvidence.sort(key=lambda x: x[3])

    #Prendo solo i primi 5
    listaEvidence = listaEvidence[:5]

    #Ho pronto id, label e evidence
    dizionario = {
            "id":rumor['id'],
            "predicted_label":prediction,
            "predicted_evidence":listaEvidence
        }
    listaDizionari.append(dizionario)

In [ ]:
listaDizionari

Salvo

In [19]:
with open("Results/Verification/NaiveBayes_verification_results.json", "w") as fr:
    json.dump(listaDizionari, fr)

In [20]:
#Scrivo il file di testo per le la verifica delle evidence
with open("Results/Evidence/NaiveBayes_evidence_results.txt", "w") as fe:
    for rumor in listaDizionari:
        #Scrivo le cose che servono per la verifica delle evidence
        i = 1
        for evid in rumor['predicted_evidence']:
            fe.write(rumor['id']+"\tQ0\t"+evid[1]+"\t"+str(i)+"\t"+str(evid[3])+"\tIO\n")
            i = i+1